In [4]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from tqdm import tqdm

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
   
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    loader = tqdm(enumerate(train_loader))
    for batch_idx, (data, target) in loader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        loader.set_postfix(loss = loss.detach().cpu().item())

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

def main():
    # Training settings
    use_cuda =  torch.cuda.is_available()


    device = torch.device("cuda" if use_cuda else "cpu")

    kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size= 64, shuffle=True, **kwargs)
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=64, shuffle=True, **kwargs)


    model = Net().to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.0001)

    for epoch in range(1, 5 + 1):
        train(model, device, train_loader, optimizer, epoch)
        test(model, device, test_loader)

       
if __name__ == '__main__':
    main()

938it [00:11, 82.08it/s, loss=0.124]  



Test set: Average loss: 0.1005, Accuracy: 9689/10000 (97%)



938it [00:11, 83.70it/s, loss=0.285]  



Test set: Average loss: 0.0617, Accuracy: 9805/10000 (98%)



938it [00:11, 82.86it/s, loss=0.0547]  



Test set: Average loss: 0.0424, Accuracy: 9864/10000 (99%)



938it [00:11, 80.81it/s, loss=0.071]   



Test set: Average loss: 0.0403, Accuracy: 9870/10000 (99%)



654it [00:08, 78.03it/s, loss=0.0579]  


KeyboardInterrupt: 